<a href="https://colab.research.google.com/github/GirumSe/DataDrive2030-Early-Learning-Predictors-Challenge-my-solution/blob/main/E_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install optuna -q
%pip install eli5 -q
%pip install catboost -q
%pip install bayesian-optimization -q
%pip install shap -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 9.9 MB/s eta 0:00:00


In [2]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
#import cupy as cp
import tensorflow as tf
import eli5
import shap
import optuna
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load files
train = pd.read_csv('/content/drive/MyDrive/early_learning/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/early_learning/Test.csv')
ss = pd.read_csv('/content/drive/MyDrive/early_learning/SampleSubmission.csv')

In [5]:
fe = []; cat_fe = []; no_fe = []
for i in train.columns[1:]:
  if train[i].isnull().sum() < 6100:
    fe.append(i)
    if train[i].dtype == 'O':
      cat_fe.append(i)
      print("There are "+ str(len(train[i].value_counts())) + " in " + str(i))
  else:
    no_fe.append(i) 

print ('----------------------------------')
print(str(len(fe)))
print(str(len(cat_fe)))
print(str(len(no_fe)))

There are 254 in child_date
There are 535 in child_enrolment_date
There are 4 in child_grant
There are 4 in child_years_in_programme
There are 4 in child_observe_attentive
There are 4 in child_observe_concentrated
There are 4 in child_observe_diligent
There are 4 in child_observe_interested
There are 2 in child_gender
There are 1018 in child_dob
There are 3 in child_stunted
There are 4 in child_age_group
There are 153 in id_mn_best
There are 10 in prov_best
There are 50 in id_dc_best
There are 50 in dc_best
There are 153 in mn_best
There are 5 in pra_free_play
There are 4 in pra_free_play_outdoor
There are 31 in pra_groupings
There are 2 in pra_groupings_1
There are 2 in pra_groupings_2
There are 2 in pra_groupings_3
There are 2 in pra_groupings_4
There are 2 in pra_groupings_5
There are 3 in pra_engaged
There are 4 in pra_agency_choice
There are 4 in pra_agency_explore
There are 4 in pra_agency_questions
There are 4 in pra_agency_understand
There are 4 in pra_agency_play
There are 4 i

In [6]:
train = train[fe]
te = test[fe[:-1]]

handeling 

In [ ]:
string = "child_gender child_dob child_zha child_stunted child_attends child_attendance child_languages child_age_group id_mn_best prov_best id_dc_best dc_best mn_best ward_best id_enumerator id_facility pqa_date pqa_class_age pqa_class_age_1 pqa_class_age_2 pqa_class_age_3 pqa_class_age_4 pqa_class_age_5 pqa_class_age_6 pqa_class_assistants pqa_environment_areas pqa_environment_variety pqa_environment_appropriate pqa_environment_accessible pqa_environment_open pqa_environment_outdoor pqa_score_environment pqa_assessment_observation pqa_assessment_systematic pqa_score_assessment pqa_relationships_peers pqa_relationships_staff pqa_relationships_acknowledge pqa_relationships_discipline pqa_score_relationships pqa_curriculum_ncf pqa_curriculum_plan pqa_curriculum_balance pqa_curriculum_numeracy pqa_curriculum_literacy pqa_score_curriculum pqa_teaching_choice pqa_teaching_engagement pqa_teaching_participation pqa_teaching_questions pqa_teaching_support pqa_score_teaching pqa_class pra_free_play pra_free_play_outdoor pra_groupings pra_groupings_1 pra_groupings_2 pra_groupings_3 pra_groupings_4 pra_groupings_5 pra_engaged pra_agency_choice pra_agency_explore pra_agency_questions pra_agency_understand pra_agency_play pra_agency_learn pra_agency_order pra_plans pra_plans_1 pra_plans_2 pra_plans_3 pra_plans_0 pra_gender pra_experience pra_class_present pra_date pra_class_size pra_class_attendance pra_class_attendance_precovid pra_class_size_large pra_cohort pra_cohort_0 pra_cohort_1 pra_cohort_2 pra_cohort_3 pra_cohort_4 pra_cohort_5 pra_cohort_6 pra_class_language pra_plan_4yrs pra_plan_4yrs_1 pra_plan_4yrs_2 pra_plan_4yrs_3 pra_plan_4yrs_4 pra_plan_4yrs_97 pra_plan_ncf pra_plan_5yrs pra_plan_5yrs_1 pra_plan_5yrs_2 pra_plan_5yrs_3 pra_plan_5yrs_4 pra_plan_5yrs_5 pra_plan_5yrs_97 pra_plan_approved pra_plan_4yrsother pra_plan_5yrsother pra_job pra_clearance_police pra_clearance_ncp pra_salary pra_paid pra_breadwinner pra_hhsize pra_education pra_qualification pra_qualification_1 pra_qualification_2 pra_qualification_3 pra_qualification_4 pra_qualification_5 pra_qualification_6 pra_qualification_7 pra_qualification_97 pra_qualification_0 pra_educationother pra_qualificationother pra_previous pra_ncf_training pra_ncf_trainer pra_ncf_trainer_1 pra_ncf_trainer_2 pra_ncf_trainer_3 pra_ncf_trainer_4 pra_ncf_trainer_5 pra_ncf_trainer_97 pra_ncf_trainerother pra_learnership pra_training pra_training_1 pra_training_2 pra_training_3 pra_training_4 pra_training_5 pra_training_6 pra_training_0 pra_special_training pra_special_referrals pra_online_training pra_online_training_details pra_motivate_support	pra_motivate_recognition	pra_motivate_mentoring	pra_shape obs_area_1	obs_area_2	obs_area_3	obs_area_4	obs_area_5	obs_area_6	obs_area_7	obs_area_8	obs_area_0	obs_materials	obs_materials_1	obs_materials_2	obs_materials_3	obs_materials_4	obs_materials_5	obs_materials_6	obs_materials_7	obs_materials_8	obs_materials_9	obs_materials_10	obs_materials_11	obs_materials_12	obs_materials_13	obs_materials_14	obs_materials_15	obs_materials_16	obs_materials_17	obs_materials_18	obs_materials_19	obs_materials_20	obs_materials_97	obs_materials_0	obs_materialsother	obs_accessible	obs_books	obs_books_age	obs_heating_census	obs_lighting_census	obs_cooking_census	obs_water	obs_waterother	obs_potable	obs_handwashing	obs_handwashing_1	obs_handwashing_2	obs_handwashing_3	obs_handwashing_0	obs_handwashing_97	obs_toilet	obs_toilet_1	obs_toilet_2	obs_toilet_3	obs_toilet_4	obs_toilet_5	obs_toilet_6	obs_toilet_7	obs_toilet_8	obs_toilet_0	obs_toilet_97	obs_toilets_children	obs_toilets_gender	obs_building	obs_shared	obs_outdoor	obs_equipment	obs_equipment_0	obs_equipment_1	obs_equipment_2	obs_equipment_3	obs_equipment_4	obs_equipment_5	obs_equipment__1	obs_condition_equipment	obs_fence	obs_gate	obs_access	obs_date	obs_water_running	obs_water_running_none	obs_handwashingother	obs_handwashing_friendly	obs_toilet_clean	obs_toilet_paper	obs_heating	obs_lighting	obs_cooking	obs_electricity_working	obs_access_disability_1	obs_access_disability_2	obs_access_disability_3	obs_access_disability_4	obs_access_disability_5	obs_access_disability_6	obs_access_disability_0	obs_safety	obs_safety_1	obs_safety_2	obs_safety_3	obs_safety_4	obs_safety_5	obs_safety_6	obs_safety_7	obs_safety_8	obs_safety_9	obs_safety_10	obs_safety_0	obs_menu_display	obs_menu_compliance	obs_menu_same	obs_material_display	obs_hazard	obs_hazard_1	obs_hazard_2	obs_hazard_3	obs_hazard_4	obs_hazard_5	obs_hazard_6	obs_hazard_7	obs_hazard_8	obs_hazard_97	obs_hazard_0	obs_fencing_play_area	id_ward	id_prov	count_register_all	count_staff_all	count_children_present	count_children_attendance	count_children_precovid	gps	latitude	longitude	count_register_gender_female	count_register_gender_male	count_register_gender_other	count_register_gender	count_register_year_2021	count_register_year_2020	count_register_year_2019	count_register_year_2018	count_register_year_2017	count_register_year_2016	count_register_year_2015	count_register_year_2014	count_register_year_2013	count_register_year_school	count_register_year_grader	grade_r	count_register_race_african	count_register_race_coloured	count_register_race_indian	count_register_race_white	count_register_race_other	count_register_race	count_staff_salary_paid	count_staff_salary_unpaid	count_staff_salary	count_staff_gender_female	count_staff_gender_male	count_staff_gender_other	count_staff_gender	count_staff_contract	count_staff_time_full	count_staff_time	professionals_practitioners	count_staff_qual_skills	count_staff_qual_nqf4_5	count_staff_qual_nqf6_9	certificate_registration_partial	certificate_registration_program	certificate_registration_npo	certificate_register	count_toilets_children	count_toilets_adults	census	urban	id_team	phase_natemis	quintile	obs_access_disability	opening_hours	count_register_foreign	count_present_girls	count_present_boys	count_staff_contract_permanent	count_staff_contract_temporary	count_staff_contract_substitute	count_staff_time_part	count_staff_paid_practitioners	count_staff_paid_assistants	count_staff_paid_cooks	count_staff_paid_maintenance	count_staff_paid_managers	count_staff_paid_support	count_staff_paid	count_practitioners_all	count_practitioners_age_0	count_practitioners_age_1	count_practitioners_age_2	count_practitioners_age_3	count_practitioners_age_4	count_practitioners_age_5	count_practitioners_age_6	health	healthother	practitioner	other_practitioner	plan	language_child	language_assessment	facility_type	sanitation_educators	sanitation_learners	positionother	positionotherreason	sef_ind	language_match	elp_ind	gps_ind	pre_covid	ses_proxy	quintile_used"
cat1 = string.split()
cat2 = ['pri_separate','pri_toys', 'pri_aftercare', 'pri_facilities', 'pri_land', 'pri_facilitiesother', 'pri_landother', 'pri_fundingother', 'pri_bank', 'pri_transport', 'pri_meal_1', 'pri_meal_3', 'pri_meal_4', 'pri_meal_2', 'pri_registered_partial', 'pri_registered_programme', 'pri_registered_npo', 'pri_registered_dsd', 'pri_subsidy', 'pri_network', 'pri_reason_register_year', 'pri_attendance_usual', 'pri_precovid_attendance', 'pri_kitchen', 'pri_funding_6', 'pri_funding_7', 'pri_funding_donations', 'pri_funding_97', 'pri_attendance', 'pri_fees_exceptions', 'pri_education', 'pri_qualification_1', 'pri_qualification_2', 'pri_qualification_3', 'pri_qualification_4', 'pri_qualification_5', 'pri_qualification_6', 'pri_qualification_7', 'pri_qualification_97', 'pri_qualification_0', 'pri_qualificationother', 'pri_founder', 'pri_founderother', 'pri_same_language', 'pri_location', 'pri_locationother', 'pri_library', 'pri_dsd_conditional', 'pri_dsd_conditional_other', 'pri_dsd_unregistered_other', 'pri_registered_health', 'pri_zoning', 'pri_registered_cipc', 'pri_network_type', 'pri_network_type_1', 'pri_network_type_2', 'pri_network_type_3', 'pri_network_type_97', 'pri_name_network_forum', 'pri_email_network_forum', 'pri_name_network_ngo', 'pri_name_network_alliance', 'pri_name_network_other', 'pri_children_4_6_years', 'pri_difficult_see', 'pri_difficult_hear', 'pri_difficult_walk', 'pri_difficult_hold', 'pri_difficult_communicate', 'pri_difficult_learn', 'pri_support_dsd', 'pri_support_dbe', 'pri_support_municipality', 'pri_support_ngo', 'pri_firstaid', 'pri_staff_employed', 'pri_staff_changes_reasons', 'pri_staff_changes_reasonsother', 'pri_covid_staff_retrench', 'pri_covid_staff_salaries', 'pri_money', 'pri_money_1', 'pri_money_2', 'pri_money_3', 'pri_money_97', 'pri_money__1', 'pri_funding_1', 'pri_funding_2', 'pri_funding_3', 'pri_funding_4', 'pri_funding_5', 'pri_funding_none', 'pri_moneyother']

In [ ]:
categorical_cols = cat1 + cat2
for col in categorical_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    mode_val = train[col].mode()[0] # get the mode value of the column
    train[col].fillna(mode_val, inplace=True) # fill null values with the mode value

In [ ]:
categorical_cols = cat1 + cat2
for col in categorical_cols:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col].astype(str))

In [ ]:
cols_to_fill = ['pri_amount_funding_fees',
                'pri_amount_funding_dsd',
                'pra_measure_rectangle_width',
                'sanitation_educators',
                'pri_fees_amount_2_3',
                'count_staff_paid_practitioners',
                'count_staff_paid',
                'pri_fees_amount_4_6',
                'count_staff_paid_cooks',
                'pra_measure_rectangle_length',
                'count_staff_paid_maintenance',
                'count_staff_paid_assistants',
                'count_staff_paid_support',
                'count_staff_paid_managers',
                'pri_fees_amount_0_1']

# fill null values in each column with its mode value
for col in cols_to_fill:
    mode_val = train[col].mode()[0] # get the mode value of the column
    train[col].fillna(mode_val, inplace=True) # fill null values with the mode value

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Create an instance of IterativeImputer
imputer = IterativeImputer()

# Fit and transform the imputer on your dataframe with the selected columns
train[['child_observe_total','child_height'
,
'language_match',
'quintile','teacher_social_total']] = imputer.fit_transform(train[['child_observe_total','child_height'
                                                                    ,
                                                                    'language_match',
                                                                    'quintile','teacher_social_total']])


In [7]:
# Find the numeric columns in the dataframe
numeric_cols = train.select_dtypes(include='number').columns

In [8]:
from scipy.stats.mstats import winsorize

# Set the winsorization limits
limits = [0.05, 0.05]
# Iterate over each numeric column
for column in train.columns:
    if column != 'target' and column in numeric_cols:
        column_values = train[column].values
        winsorized_values = winsorize(column_values, limits)
        if not (column_values == winsorized_values).all():
            print(f"Number of extreme values in {column}: {len(column_values) - len(winsorized_values)}")
            train[column] = winsorized_values

Number of extreme values in child_age: 0
Number of extreme values in child_months_enrolment: 0
Number of extreme values in child_height: 0
Number of extreme values in child_observe_total: 0
Number of extreme values in child_zha: 0
Number of extreme values in ward_best: 0
Number of extreme values in id_enumerator: 0
Number of extreme values in id_facility: 0
Number of extreme values in pri_days: 0
Number of extreme values in pri_time_open_hours: 0
Number of extreme values in pri_time_open_minutes: 0
Number of extreme values in pri_time_close_hours: 0
Number of extreme values in pri_time_close_minutes: 0
Number of extreme values in pri_year: 0
Number of extreme values in pri_fees_amount: 0
Number of extreme values in pri_capacity: 0
Number of extreme values in pri_fees_amount_pv: 0
Number of extreme values in pri_funding_subsidy: 0
Number of extreme values in teacher_duration: 0
Number of extreme values in teacher_social_total: 0
Number of extreme values in teacher_selfcare_total: 0
Numb

In [9]:
x = train[train.columns[:-1]]
y = train[train.columns[-1]]
x = x.fillna('')
y = y.fillna('')
te = te.fillna('')

Spliting train and test data

In [10]:
# Split data into training and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42, shuffle = True)

In [11]:
train_dataset = Pool(data = X_train, label = y_train, cat_features=cat_fe)
test_dataset = Pool(data = X_test, label = y_test, cat_features=cat_fe)

MODELS TO TRAIN

In [ ]:
# Define the objective function to optimize
def catboost_objective(trial):
    # Define the hyperparameter search space
    params = {
        'iterations': trial.suggest_int('iterations', 5000, 40000),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'l2_leaf_reg': trial.suggest_uniform('l2_leaf_reg', 0, 10)
    }

    # Create the model with the specified hyperparameters
    cmod = CatBoostRegressor(
        **params,
        random_seed=42,
        task_type="GPU",
        verbose=2000
    )
    cmod.fit(train_dataset, eval_set=test_dataset, early_stopping_rounds=300, use_best_model=True, verbose=0)

    # Evaluate the model on the testing data
    test_pred = cmod.predict(X_test)
    rmse = mean_squared_error(y_test, test_pred, squared=False)

    return rmse

# Run the Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(catboost_objective, n_trials=15)

# Get the best hyperparameters
best_params = study.best_params
n_estimators = best_params['iterations']
learning_rate = best_params['learning_rate']
max_depth = best_params['max_depth']
l2_leaf_reg = best_params['l2_leaf_reg']

# Train the model with the best hyperparameters
cmod = CatBoostRegressor(
    iterations=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    l2_leaf_reg=l2_leaf_reg,
    task_type="GPU",
    random_seed=42,
    verbose=300
)
cmod.fit(train_dataset, eval_set=test_dataset, early_stopping_rounds=300, use_best_model=True)

# Evaluate the model on the testing data
test_pred = cmod.predict(X_test)
rmse = mean_squared_error(y_test, test_pred, squared=False)
print("RMSE:", rmse)

[I 2023-04-11 01:39:44,700] A new study created in memory with name: no-name-48a42230-219c-4ce2-b6c9-0d55c4e92957
[I 2023-04-11 01:51:23,501] Trial 0 finished with value: 9.985847311305509 and parameters: {'iterations': 13647, 'learning_rate': 0.07024290534593859, 'max_depth': 4, 'l2_leaf_reg': 7.8923437827669645}. Best is trial 0 with value: 9.985847311305509.
[I 2023-04-11 02:13:42,567] Trial 1 finished with value: 9.98317038711354 and parameters: {'iterations': 14543, 'learning_rate': 0.04230036015080533, 'max_depth': 7, 'l2_leaf_reg': 7.985243133425178}. Best is trial 1 with value: 9.98317038711354.
[I 2023-04-11 02:23:44,898] Trial 2 finished with value: 10.022113879386366 and parameters: {'iterations': 14742, 'learning_rate': 0.06260642232835263, 'max_depth': 3, 'l2_leaf_reg': 7.5550304493291565}. Best is trial 1 with value: 9.98317038711354.
[I 2023-04-11 02:36:22,377] Trial 3 finished with value: 9.947437927494798 and parameters: {'iterations': 30511, 'learning_rate': 0.0801413

In [27]:
params = {'iterations': 8.628e+03, 'learning_rate': 0.07379, 'max_depth': 7, 'l2_leaf_reg': 2.162}
cmod = CatBoostRegressor(**params, random_seed = 42, verbose = 300)
cmod.fit(train_dataset, eval_set = test_dataset, early_stopping_rounds = 300, use_best_model = True)

0:	learn: 14.7600617	test: 14.9324904	best: 14.9324904 (0)	total: 1.5s	remaining: 3h 35m 14s
300:	learn: 8.7961470	test: 10.2023388	best: 10.2007230 (295)	total: 3m 22s	remaining: 1h 33m 15s
600:	learn: 7.9115852	test: 10.0371460	best: 10.0352779 (590)	total: 6m 39s	remaining: 1h 28m 58s
900:	learn: 7.2658263	test: 9.9866202	best: 9.9810872 (859)	total: 9m 55s	remaining: 1h 25m 4s
1200:	learn: 6.6922277	test: 9.9773703	best: 9.9630020 (1014)	total: 13m 14s	remaining: 1h 21m 52s
1500:	learn: 6.1852308	test: 9.9555268	best: 9.9555268 (1500)	total: 16m 32s	remaining: 1h 18m 31s
1800:	learn: 5.7761039	test: 9.9318238	best: 9.9295995 (1760)	total: 19m 48s	remaining: 1h 15m 4s
2100:	learn: 5.4158434	test: 9.9257619	best: 9.9175771 (1949)	total: 23m 4s	remaining: 1h 11m 42s
2400:	learn: 5.0700519	test: 9.9007355	best: 9.8994964 (2381)	total: 26m 23s	remaining: 1h 8m 26s
2700:	learn: 4.7337247	test: 9.8826221	best: 9.8781658 (2684)	total: 29m 41s	remaining: 1h 5m 9s
Stopped by overfitting dete

In [28]:
predc = cmod.predict(te)
testc = cmod.predict(X_test)
mean_squared_error(y_test, testc, squared=False)

9.878626215237876

In [23]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
# Create training and testing datasets
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_fe)
test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=cat_fe)
# Train the model
lmod = lgb.train(params, train_data, valid_sets=[test_data], early_stopping_rounds=300, verbose_eval=300)

# Make predictions on the testing dataset
testl = lmod.predict(X_test, num_iteration=lmod.best_iteration)

# Evaluate the model performance
mse = mean_squared_error(y_test, testl, squared=False)

print('Root Mean Squared Error: ', mse)

ValueError: ignored

In [29]:
expa = shap.TreeExplainer(cmod)
shap_val = expa.shap_values(te)

In [30]:
features = te.columns

In [31]:
f = {
    'f1': [],'f2' : [],'f3' : [],'f4' : [],'f5' : [],'f6' : [],'f7' : [],'f8' : [],'f9' : [],'f10' : [],'f11' : [],'f12' : [],'f13' : [],'f14': [],'f15' : []
}

In [32]:
for shap_vals in shap_val:
  arr = np.argsort(shap_vals)[::-1][:15]
  for ind, a in enumerate(arr):
    name_f = f'f{ind+1}'
    f[name_f].append(features[a])

In [33]:
ss['target'] = predc
ss['feature_1'] = f['f1']
ss['feature_2'] = f['f2']
ss['feature_3'] = f['f3']
ss['feature_4'] = f['f4']
ss['feature_5'] = f['f5']
ss['feature_6'] = f['f6']
ss['feature_7'] = f['f7']
ss['feature_8'] = f['f8']
ss['feature_9'] = f['f9']
ss['feature_10'] = f['f10']
ss['feature_11'] = f['f11']
ss['feature_12'] = f['f12']
ss['feature_13'] = f['f13']
ss['feature_14'] = f['f14']
ss['feature_16'] = f['f15']

In [34]:
ss

,child_id,target,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16
0,ID_0I0999N6S,54.405937,child_observe_diligent,child_observe_attentive,teacher_emotional_total,child_observe_interested,child_observe_total,child_age,child_years_in_programme,child_gender,teacher_emotional_met,id_team,pra_ind,teacher_emotional_selfstarter,latitude,teacher_social_peers,feature,count_children_present
1,ID_GQ6ONJ4FP,45.493638,teacher_emotional_total,child_observe_attentive,child_observe_diligent,child_observe_concentrated,child_observe_interested,child_observe_total,teacher_emotional_met,pri_attendance,id_facility,count_children_present,teacher_emotional_understand,teacher_social_peers,child_gender,pra_agency_choice,feature,count_register_year_2015
2,ID_YZ76CVRW3,44.717774,id_enumerator,child_observe_diligent,child_observe_attentive,child_age,prov_best,child_date,mn_best,teacher_emotional_selfstarter,pri_fees_amount,obs_space,id_team,obs_materials,teacher_emotional_understand,ses_proxy,feature,id_prov
3,ID_BNINCRXH8,68.424854,child_observe_attentive,child_observe_diligent,child_observe_concentrated,child_age,prov_best,id_enumerator,child_observe_total,language_assessment,child_stunted,teacher_duration,child_gender,teacher_emotional_met,child_years_in_programme,teacher_social_assistance,feature,language_child
4,ID_1U7GDTLRI,46.674002,child_observe_diligent,child_observe_interested,id_facility,child_observe_attentive,pri_attendance,id_ward,id_dc_best,child_observe_total,pri_meal_1,prov_best,pra_agency_questions,longitude,teacher_emotional_understand,id_prov,feature,obs_materials_8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,ID_LBPQ2VMQZ,50.513045,child_age_group,child_age,child_observe_interested,child_observe_attentive,child_observe_concentrated,teacher_social_nonaggressive,id_ward,teacher_emotional_selfstarter,teacher_emotional_understand,child_observe_total,child_date,child_stunted,teacher_emotional_met,child_years_in_programme,feature,pri_bank
3676,ID_H2RKKMMKK,52.667126,child_age_group,child_age,id_enumerator,ward_best,prov_best,child_height,child_zha,child_date,teacher_emotional_understand,child_gender,ses_proxy,obs_handwashing_1,id_team,latitude,feature,count_register_year_2020
3677,ID_VY8KX7YTZ,41.687408,child_observe_diligent,child_observe_attentive,child_age,child_observe_concentrated,child_observe_interested,pri_attendance,child_observe_total,child_gender,teacher_emotional_adjust,child_dob,obs_space,teacher_emotional_selfstarter,obs_materials,obs_equipment__1,feature,count_register_year_2020
3678,ID_EO2MYZ4M7,38.783792,id_enumerator,id_facility,pri_language_7,language_child,teacher_social_initiative,obs_handwashing_1,count_children_present,teacher_social_assistance,child_gender,id_dc_best,child_zha,child_grant,obs_space,ward_best,feature,count_register_year_2020


In [35]:
ss.to_csv('BaselineSubmission47.csv', index=False)